In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Train/Val/Test - 0.65:0.20:0.15
# CPU - Intel(R) Xeon(R) CPU @ 2.30GHz
# GPU - Tesla K80

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, BatchNormalization, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import decode_predictions, preprocess_input, ResNet50
import pandas as pd
import numpy as np
import seaborn as sn

In [ ]:
img_gen = ImageDataGenerator(preprocessing_function = preprocess_input, horizontal_flip = True, vertical_flip = True)

In [ ]:
train_data = img_gen.flow_from_directory("/content/drive/MyDrive/Plastics", target_size = (224,224), class_mode = "binary", batch_size = 32)

Found 76 images belonging to 2 classes.


In [ ]:
base_model = ResNet50(include_top = False, weights = "imagenet", input_shape = (224,224,3))

for layer in base_model.layers :
  layer.trainable = False

for layer in base_model.layers[-1:] :
  layer.trainable = True

x = base_model.output
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(128, activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
pred = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = base_model.input, outputs = pred)

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
import tensorflow as tf

with tf.device("/gpu:1") :
  history = model.fit(train_data, epochs = 15)

Epoch 1/15


In [ ]:
model.evaluate(train_data)

48/48 [==============================] - 11s 221ms/step - loss: 0.0044 - accuracy: 1.0000


[0.004366893786936998, 1.0]

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread("/content/drive/MyDrive/Plastics/negative/img1 (390).jpg")
#img = cv2.imread("/content/drive/MyDrive/Plastics/negative/img1 (129).jpg")
mod_img = cv2.resize(img, (224,224), interpolation = cv2.INTER_CUBIC)
result = model.predict(np.array([mod_img])) # 1 - plastic / 0 - no plastic or sea bed
test_img = mod_img.copy()

if round(result[0][0]) == 1:
  test_img = cv2.putText(test_img, "Plastic found", (100,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
else :
  test_img = cv2.putText(test_img, "Sea bed", (100,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)

test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
plt.imshow(test_img)
plt.savefig("test_img.jpg")